In [ ]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import dit_fast_attention
import os
file_path=os.path.abspath(dit_fast_attention.__file__)
dir_path=os.path.dirname(file_path)
os.chdir(dir_path)
from diffusers import DiTPipeline, DPMSolverMultistepScheduler
import argparse
import torch
from evaluation import test_latencies
from dit_fast_attention import transform_model_fast_attention
import numpy as np
import os
import time
import thop
from utils import calculate_flops
import re
import seaborn as sns
from collections import namedtuple
os.environ["https_proxy"]="http://10.10.20.100:1089"


In [ ]:
pipe = DiTPipeline.from_pretrained("facebook/DiT-XL-2-512", torch_dtype=torch.float16).to("cuda")
n_calib=8
n_steps=20
calib_x=torch.randint(0, 1000, (n_calib,),generator=torch.Generator().manual_seed(3)).to("cuda")


In [ ]:
from torch import tensor
def parse_txt_data(txt_data):
    data = []
    for block in txt_data.split('\n\n'):
        lines = block.strip().split('\n')
        for line in lines:
            if line.startswith('Namespace'):
                namespace = re.findall(r"Namespace\((.*)\)", line)[0]
                namespace_args = [arg.split('=') for arg in namespace.split(', ')]
                namespace_dict = {arg[0]: eval(arg[1]) for arg in namespace_args}
                # namespace_obj = namedtuple('Namespace', namespace_dict.keys())(**namespace_dict)
                data.append(namespace_dict)
            elif line.startswith('calib_ssim'):
                calib_ssim = float(line.split('=')[1])
                # data.append(calib_ssim)
                namespace_dict["calib_ssim"] = calib_ssim
            elif line.startswith('{'):
                metrics = eval(line)
                # data.append(metrics)
                namespace_dict["metrics"] = metrics
            elif line.startswith('macs'):
                macs = eval(line.split('=')[1])
                namespace_dict["macs"] = macs
                # data.append(macs)
            elif line.startswith('attn_mac'):
                attn_mac = float(line.split('=')[1])
                # data.append(attn_mac)
                namespace_dict["attn_mac"] = attn_mac
            elif line.startswith('latencies'):
                latencies = eval(line.split('=')[1])
                # data.append(latencies)
                namespace_dict["latencies"] = latencies

                # print(latencies)
    return data

txt_data=open("output/results.txt").read()
data=parse_txt_data(txt_data)


In [ ]:
iss=[]
fids=[]
attn_macs=[]
seqs=[0.975,0.95,0.925,0.9,0.875,0.85]
visited=[]
for l in data[::-1]:
    if l["model"]=="facebook/DiT-XL-2-512" and l["n_calib"]==8 and l["eval_n_images"]==5000 and "attn_mac" in l and l["threshold"] in seqs:
        if "metrics" not in l or not "IS" in l["metrics"]:
            continue
        if l["threshold"] in visited:
            continue
        visited.append(l["threshold"])
        iss.append(l["metrics"]["IS"][0])
        fids.append(l["metrics"]["FID"])
        attn_macs.append(l["attn_mac"])

In [ ]:
# sort
import seaborn as sns
sns.set_style("whitegrid")

fig=plt.figure(figsize=(3.5,3.5))

inds = np.argsort(attn_macs)[::-1]
iss = np.array(iss)[inds]
attn_macs = np.array(attn_macs)[inds]
fids = np.array(fids)[inds]

raw_macs=2705
raw_fid=30.868
raw_is=210.0112

plt.scatter(attn_macs, iss,label="DiTFastAttn (IS)")
plt.scatter(raw_macs, raw_is, marker='d',label="Raw Attention (IS)")

# annotate points D1 D2 ... D6
for i in range(6):
    plt.annotate(f"D{i+1}", (attn_macs[i], iss[i]), textcoords="offset points", xytext=(0,-20), ha='center')


plt.scatter(attn_macs, fids, label="DiTFastAttn (FID)")
plt.scatter(raw_macs, raw_fid, marker='d',label="Raw Attention (FID)")

plt.xlabel("Attention MACs (GFLOPs)")
plt.legend()

# plt.scatter(attn_macs, fids)




In [ ]:
# PLOT!!


seqs=[0.975,0.95,0.925,0.9,0.875,0.85]

dir_raw="output/old/facebook_DiT-XL-2-512_steps20"
dir_fast="output/facebook_DiT-XL-2-512_calib8_steps20_threshold{}_window128_sequentialFalse"
# dir_fast2="output/facebook_DiT-XL-2-512_calib8_steps20_threshold0.85_window128_sequentialFalse"
for filename in os.listdir(dir_raw)[:300:12]:
    if filename.endswith(".png"):
        print(filename)
        file_raw=os.path.join(dir_raw,filename)
        file_fast=os.path.join(dir_fast,filename)
        img_raw=Image.open(file_raw)
        img_concat=Image.new('RGB', (img_raw.width*(len(seqs)+1), img_raw.height))
        img_concat.paste(img_raw, (0, 0))
        for t in seqs:
            img_fast=Image.open(os.path.join(dir_fast.format(t),filename))
            img_concat.paste(img_fast, (img_raw.width*(seqs.index(t)+1), 0))
        
        # img_concat=Image.new('RGB', (img_raw.width + img_fast.width, img_raw.height))
        # img_concat.paste(img_raw, (0, 0))
        # img_concat.paste(img_fast, (img_raw.width, 0))
        # resize to 30%
        img_concat=img_concat.resize((int(img_concat.width*0.5),int(img_concat.height*0.5)))
        img_concat.show()
        
        

In [ ]:
# PixArt-1K


iss=[]
fids=[]
clip_scores=[]
attn_macs=[]
seqs=[0.975,0.95,0.925,0.9,0.875,0.85]
visited=[]
for l in data[::-1]:
    if l["model"]=="PixArt-alpha/PixArt-Sigma-XL-2-1024-MS" and l["n_calib"]==4 and l["eval_n_images"]==5000 and "attn_mac" in l and l["threshold"] in seqs:
        if "metrics" not in l or not "IS" in l["metrics"]:
            continue
        if l["threshold"] in visited:
            continue
        visited.append(l["threshold"])
        iss.append(l["metrics"]["IS"][0])
        fids.append(l["metrics"]["FID"])
        attn_macs.append(l["attn_mac"])
        clip_scores.append(l["metrics"]["CLIP"])
# sort
sns.set_style("whitegrid")

fig=plt.figure(figsize=(3.5,3.5))

inds = np.argsort(attn_macs)[::-1]
iss = np.array(iss)[inds]
attn_macs = np.array(attn_macs)[inds]
fids = np.array(fids)[inds]
clip_scores = np.array(clip_scores)[inds]

raw_macs=43293.27034368
raw_is=22.6652
raw_fid=84.1625
raw_clip_score=31.9019

plt.scatter(attn_macs-raw_macs, iss,label="DiTFastAttn (IS)")
plt.scatter(raw_macs, raw_is, marker='d',label="Raw Attention (IS)")

# annotate points D1 D2 ... D6
for i in range(len(iss)):
    plt.annotate(f"D{i+1}", ((attn_macs-raw_macs)[i], fids[i]), textcoords="offset points", xytext=(0,-20), ha='center')


plt.scatter(attn_macs-raw_macs, fids, label="DiTFastAttn (FID)")
plt.scatter(raw_macs, raw_fid, marker='d',label="Raw Attention (FID)")

plt.scatter(attn_macs-raw_macs, clip_scores, label="DiTFastAttn (CLIP)")
plt.scatter(raw_macs, raw_clip_score, marker='d',label="Raw Attention (CLIP)")

plt.xlabel("Attention MACs (GFLOPs)")
plt.legend()

# plt.scatter(attn_macs, fids)




In [ ]:
visited

In [ ]:
# PLOT!!


seqs=visited

dir_raw="output/PixArt/PixArt-alpha_PixArt-Sigma-XL-2-1024-MS_steps20"
dir_fast="output/PixArt-alpha_PixArt-Sigma-XL-2-1024-MS_calib4_steps20_threshold{}_window256_seqFalse"
# dir_fast2="output/facebook_DiT-XL-2-512_calib8_steps20_threshold0.85_window128_sequentialFalse"
for filename in os.listdir(dir_raw)[:300:10]:
    if filename.endswith(".jpg"):
        print(filename)
        file_raw=os.path.join(dir_raw,filename)
        file_fast=os.path.join(dir_fast,filename)
        img_raw=Image.open(file_raw)
        img_concat=Image.new('RGB', (img_raw.width*(len(seqs)+1), img_raw.height))
        img_concat.paste(img_raw, (0, 0))
        for t in seqs:
            path=os.path.join(dir_fast.format(t),filename)
            if os.path.exists(path):
                img_fast=Image.open(path)
                img_concat.paste(img_fast, (img_raw.width*(seqs.index(t)+1), 0))
        
        # img_concat=Image.new('RGB', (img_raw.width + img_fast.width, img_raw.height))
        # img_concat.paste(img_raw, (0, 0))
        # img_concat.paste(img_fast, (img_raw.width, 0))
        # resize to 30%
        img_concat=img_concat.resize((int(img_concat.width*0.25),int(img_concat.height*0.25)))
        img_concat.show()
        
        